In [7]:
from time import time
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import numpy as np
import cv2
from tqdm import tqdm

eps = 1e-5

pixel_width = 320
pixel_height = 180
aspect_ratio = pixel_width / pixel_height
theta = np.pi / 180 * 30
G = 6.673e-11
c = 299792458
M = 1.98892e30 * 1e5
rin = 3 * G * M / (c ** 2)
rout = 5 * G * M / (c ** 2)

def get_eye_unit_vector (x, y):
  ret = np.array([x - pixel_width // 2, y - pixel_height // 2, -np.minimum(pixel_width, pixel_height) // 2], dtype=np.float64)
  return ret / np.linalg.norm(ret)

def get_rotate_x_mat (theta):
  return np.array([
    [1, 0, 0],
    [0, np.cos(theta), np.sin(theta)],
    [0, -np.sin(theta), np.cos(theta)],
  ], dtype=np.float64)

def get_rotate_z_mat_by_pixel (x, y):
  if x == 0 and y == 0:
    return np.array([
      [1, 0, 0],
      [0, 1, 0],
      [0, 0, 1],
    ], dtype=np.float64)
  xy_unit_vec = np.array([x, y], dtype=np.float64) / np.linalg.norm(np.array([x, y], dtype=np.float64))
  return np.array([
    [xy_unit_vec[1], xy_unit_vec[0], 0],
    [-xy_unit_vec[0], xy_unit_vec[1], 0],
    [0, 0, 1],
  ], dtype=np.float64)

def get_accretion_plane (theta, pixel_x, pixel_y):
  ret = (
    np.array([0, 1, 0], dtype=np.float64)
    @ get_rotate_x_mat(theta)
    @ get_rotate_z_mat_by_pixel(pixel_x, pixel_y)
  )
  ret = ret / np.linalg.norm(ret)
  return ret

def get_accretion_direction (theta, pixel_x, pixel_y):
  n = get_accretion_plane(theta, pixel_x, pixel_y)
  ret = np.array([n[2], -n[1]], dtype=np.float64)
  return ret

def get_geodesic (M, r0, phi0, ux0, uy0, error_check_dense=1, dense=0.01):
  def f (_, x):
    return (x[1], -x[0] + 3 * G * M * (x[0] ** 2) / (c ** 2))

  r_p0 = ux0 * np.cos(phi0) + uy0 * np.sin(phi0)
  phi_p0 = (-ux0 * np.sin(phi0) + uy0 * np.cos(phi0)) / r0

  dr_dphi0 = 1 / phi_p0 * r_p0

  u0 = 1 / r0
  up0 = -dr_dphi0 / r0 ** 2

  phi_ode = phi0
  u_ode0 = [u0, up0]

  phi_step = np.arange(phi0, phi0 + np.pi * 8, error_check_dense, dtype=np.float64)

  phi_ode = [phi0]
  u_ode = [np.array(u_ode0, dtype=np.float64)]

  for i in range(len(phi_step) - 1):
    t_eval=np.arange(phi_step[i], phi_step[i + 1], dense, dtype=np.float64)
    t_eval[0] = phi_step[i]
    t_eval[-1] = phi_step[i + 1]
    start = time()
    sol = solve_ivp(f, (phi_step[i], phi_step[i + 1]), u_ode[-1], dense_output=True, t_eval=t_eval)
    end = time()
    if not sol.success:
      print(sol)
    if 1 / sol.y[0, -1] < 2 * G * M / (c ** 2) or not sol.success:
      break
    phi_ode += list(sol.t)
    u_ode += list(np.transpose(sol.y))

  phi_ode = np.array(phi_ode)
  u_ode = np.array(u_ode)

  u = u_ode[:, 0]
  r = 1 / u

  return phi_ode, u_ode

def check_cross (vec, p1, p2):
  return np.cross(vec, p1) * np.cross(vec, p2) < 0

def check_accretion_direction_cross (accretion_direction, rin, rout, pixel_x, pixel_y):
  eye = get_eye_unit_vector(pixel_x, pixel_y)
  eye_xy = np.array([eye[2], np.linalg.norm([eye[0], eye[1]])], dtype=np.float64)
  geodesic = get_geodesic(M, 800000000, 0, eye_xy[0], eye_xy[1])
  for i in range(geodesic[0].shape[0] - 1):
    r = 1 / geodesic[1][i][0]
    p0 = [r * np.cos(geodesic[0][i]), r * np.sin(geodesic[0][i])]
    p1 = [r * np.cos(geodesic[0][i + 1]), r * np.sin(geodesic[0][i + 1])]
    if check_cross(accretion_direction, p0, p1) and r >= rin and r <= rout:
      return True
  
  return False

img = np.zeros([pixel_height, pixel_width], dtype=np.uint8)
for y in tqdm(range(img.shape[0])):
  for x in range(img.shape[1]):
    # print(x, y)
    if check_accretion_direction_cross(get_accretion_direction(theta, x, y), rin, rout, x, y):
      img[y][x] = 255
    else:
      img[y][x] = 0

cv2.imwrite('accretion-plane.png', img)


# plt.axis('equal')
# plt.plot(x, y)

# def plot_black_hole_radius (M):
#   G = 6.673e-11
#   c = 299792458
#   r = 2 * G * M / (c ** 2)
#   theta = np.arange(0, 2 * np.pi, 0.01)
#   x = r * np.cos(theta)
#   y = r * np.sin(theta)
#   plt.plot(x, y)

# plot_black_hole_radius(M)
# plt.show()


  2%|▏         | 3/180 [00:19<19:00,  6.44s/it]


KeyboardInterrupt: 